# 2 LTCM Risk

### 1. Summary stats.
#### (a) For both the gross and net series of LTCM excess returns, report the mean and volatility. Since this is monthly data, scale the mean by 12, and scale the volatility by sqrt(12).
#### (b) Report the annualized Sharpe ratio, (the SR based on the annualized mean and volatility.)
#### (c) Comment on whether the mean, volatility, and Sharpe ratio seem especially high or low relative to other assets we have seen.

### 2. Using the series of net LTCM excess returns, denoted ~r^LTCM, estimate the following regression: ~rLTCM
t =  + m~rm
t + t
(a) Report  and m. Report the R2 stat.
(b) From this regression, does LTCM appear to have much exposure to the equity-market
factor, ~rm?
3. Regression-based metrics.
(a) Calculate the Treynor ratio.
(b) Calculate the Information ratio.
4. Tail risk.
(a) Calculate the 5th worst return of the sample.
(b) Calculate the mean of the worst 4 returns from the sample.
(c) Report the skewness of the return distribution. Compare to a normal distribution with
skewness of 0.
(d) Report the kurtosis of the return distribution. Compare to a normal distribution with
kurtosis of three.1